In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import numpy
import math
from scipy.stats import pearsonr

# Create CMI without smoothing function based on difference

In [2]:
def CMI_diff(x, y):
    "This function computes the Cumulative Mutual Information based on the difference between joint and marginals CE's"
    x = x - np.mean(x)
    y = y - np.mean(y)
    N = len(x)
    lattice_x = np.zeros((N, N), dtype="float_")    # Array of lower x tile coordinates
    lattice_y = np.zeros((N, N), dtype="float_")     # Array of lower y tile coordinates
    lattice_area = np.zeros((N, N), dtype="float_")   # Array of tile areas
    lattice_count = np.zeros((N, N), dtype="uint8")    # Array of tile data counts
    x_order = np.argsort(x, kind='quicksort')
    x_sorted = x[x_order]
    y_order = np.argsort(y, kind='quicksort')
    y_sorted = y[y_order]
    ind = np.arange(0, N, 1)
    ind_x_order = np.argsort(x_order, kind='quicksort') 
    ind_x = ind[ind_x_order]
    ind_y_order = np.argsort(y_order, kind='quicksort') 
    ind_y = ind[ind_y_order]
    for i in range(N):
        for j in range(N):
            lattice_x[i, j] = x_sorted[i]
            lattice_y[i, j] = y_sorted[j]
            if i < (N-1) and j < (N-1):
                lattice_area[i, j] = (x_sorted[i+1] - x_sorted[i])*(y_sorted[j+1] - y_sorted[j])
        
    for i in range(N): 
        lattice_count[ind_x[i], ind_y[i]] = 1 

    np.cumsum(lattice_count, axis=1, out=lattice_count)
    np.cumsum(lattice_count, axis=0, out=lattice_count)
    CDF_xy = lattice_count/float(N)
    CDF_x = np.arange(0, N, 1)/float(N)
    CDF_y = CDF_x
    CE_x = 0
    CE_y = 0
    for i in range(N-1):
        if CDF_x[i] != 0:
            CE_x = CE_x - (x_sorted[i+1] - x_sorted[i]) * CDF_x[i] * math.log(CDF_x[i])
        if CDF_y[i] != 0:       
            CE_y = CE_y - (y_sorted[i+1] - y_sorted[i]) * CDF_y[i] * math.log(CDF_y[i])

    CE_xy = 0
    for i in range(N):
        for j in range(N):
            if CDF_xy[i, j] != 0:
                CE_xy = CE_xy - lattice_area[i, j] * CDF_xy[i, j] * math.log(CDF_xy[i, j])
             
    # Compute the CMI based on the difference between joint and marginals CE's        
            
    CMI = CE_xy - (max(y) - np.mean(y))*CE_x - (max(x) - np.mean(x))*CE_y  
    #if CMI < 0:
    #    CMI = 0

    rho, p = pearsonr(x, y)
    
    #return (np.sign(rho)* CMI/(np.max(x)*np.max(y)))
    return (CMI/(np.max(x)*np.max(y)))

# Create CMI with smoothing function

In [3]:
#CMI smooth takes in two data vectors, and creates bxb grid
def CMI_smooth_v1(x,y,b):
    
    if len(x)==len(y):
        sizeOfVector = len(x)
        
        x_r = ((x-min(x))/(max(x)-min(x)))*b #Let x' be the vector x rescaled on the interval [0,b]
        y_r = ((y-min(y))/(max(y)-min(y)))*b #Let y' be the vector y rescaled on the interval [0,b]
        
        x=x-numpy.mean(x)
        y=y-numpy.mean(y)
        
        #smooth data by aligning it with the bXb grid
        x_p = x_r
        y_p = y_r
        for i in range(0,sizeOfVector):
            x_p[i] = numpy.ceil(x_r[i])
            y_p[i] = numpy.ceil(y_r[i])
            
        #This will create function distribution which will count the number of points at each grid square
        distribution= [[0 for i in range(b+1)] for i in range(b+1)]
        for j in range (0,b+1):
            for i in range (0, b+1): #For every upper right corner   
                counter = 0
                for k in range (0, sizeOfVector): #For every point data given
                    if (x_p[k]== i) and (y_p[k] == j): #If the point of the data matches the upper right hand corner
                        counter = counter+1 #Then add 1 to the distribution counter
                distribution[i][j] = counter
                
        #Now we count the number of points on or below each square to create PXY
        lattice_count = numpy.asarray(distribution)    # Array of tile data counts
        numpy.cumsum(lattice_count,axis=1, out=lattice_count)
        numpy.cumsum(lattice_count, axis=0, out=lattice_count)
        PXY = lattice_count/float(sizeOfVector)
        
        #Calculate CE(X,Y)
        CEXY =0 
        for i in range (1, b+1):
            for j in range (1 ,b+1):
                if (PXY[i][j] != 0):
                    CEXY = CEXY - PXY[i][j]*numpy.log(PXY[i][j]) 
                    
        #Scale CE(X,Y) back to original data interval
        CEXY= CEXY*(((max(x)-min(x))/b))*(((max(y)-min(y))/b))

        #Find Cumulative distribution function in 1D
        P = [i/(b+1) for i in range(b +1)]

        #Calculate Cumulative Entropy of X and Y (Note CE(X) = CE(Y) by symmetry)
        CEXoY = 0
        for i in range (0,b):
            if(P[i] != 0):
                CEXoY = CEXoY - P[i]*numpy.log(P[i])

        #Scale back the data to original interval
        CEX = CEXoY*(((max(x)-min(x))/b))
        CEY = CEXoY*(((max(y)-min(y))/b))
        
        CMI = CEXY - (max(y) - numpy.mean(y))*CEX - (max(x) - numpy.mean(x))*CEY 
        CMI = CMI/numpy.max(x)*numpy.max(y)     
        return CMI
        
        

 # Calculate the minimum distance between in the x and y axis function

In [4]:
def min_int(x,y):
    x_s = numpy.sort(x)
    y_s = numpy.sort(y)

    min_x = math.inf 
    min_y = math.inf
    min_xy = math.inf

    for i in range(len(x_s)-1):
        interval =  x_s[i+1] -x_s[i]
        if ( interval < min_x) and interval != 0:
            min_x = interval

    for j in range(len(y_s)-1):
        interval =  y_s[i+1] -y_s[i]
        if ( interval < min_y) and interval != 0:
            min_y = interval

    if (min_x < min_y):
        min_xy = min_x
    else:
        min_xy = min_y
        
    return min_xy


# Create random Data

In [5]:
sizeOfVector = 40
x = numpy.random.rand(sizeOfVector) #For a given vector x with random value numbers and
y = x
#y = x + numpy.random.rand(sizeOfVector)*0.01
#y = numpy.random.rand(sizeOfVector) #a given vector y with random value numbers


#x = numpy.array([ 0.6358171 ,  0.99577192 , 0.64413589 , 0.68141891 , 0.64693171 , 0.27456281
# , 0.08515774 , 0.24606362 , 0.79079224 , 0.12927175])

#y = numpy.array([ 0.41911051 , 0.05476291 , 0.7635048 ,  0.05124239 , 0.7299192  , 0.1485072
# , 0.92069862 , 0.85522989 , 0.88012141 , 0.6219437])

# Compare CMI vs CMI Smooth

In [6]:
CMI_diff(x,y)

0.22469075143924164

In [7]:
print(math.ceil(1/min_int(x,y)))
CMI_smooth_v1(x,y, math.ceil(1/min_int(x,y)))

3537


0.0498850667586868

In [8]:
CMI_smooth_v1(x,y, 100)

0.049539579357774033

In [9]:
CMI_smooth_v1(x,y,1000)

0.049817479682909796

In [10]:
x1 = numpy.array([ 0.6358171 ,  0.99577192 , 0.64413589 , 0.68141891 , 0.64693171 , 0.27456281
 , 0.08515774 , 0.24606362 , 0.79079224 , 0.12927175])

y1 = numpy.array([ 0.41911051 , 0.05476291 , 0.7635048 ,  0.05124239 , 0.7299192  , 0.1485072
 , 0.92069862 , 0.85522989 , 0.88012141 , 0.6219437])

CMI_diff(x1,y1)

-0.23174413281629952